# Incidence Matrices for quasi-cords

## Importing modules

In [1]:
import Permutations as pm
from QuasiCord import *

from copy import copy
# import numpy as np
# import sympy
# sympy.init_printing()
# import itertools
# from functools import reduce

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=False)
# from ipywidgets import interactive, HBox, VBox
# from plotly import tools

# from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'Permutations'

## Create an instance of Square class.

In [ ]:
sqr = square_random_generator(verbose=True)
#sqr = Square((5,1,1,1)) #1,4,1,1)) #0,1,0,0)) #0,3,0,3)) #8,4,6,2)) #3,2,1,1)) #(3,4,5,2)) #(3,7,4,2)) #(1,3,1,1)) #(3,4,2,1)) #

print(sqr.code)

### Parts of a square

End points on the sides

In [ ]:
for side in sqr.sides.values():
    print("side {} --> {}".format(side.label, side.points))

Segments

In [ ]:
all_segs = [seg.ends for seg in sqr.segments]
print(all_segs)

### Graphic

In [ ]:
fig = sqr.graphic(figsize=(500,500))
fig.show()

### Incidence Matrix

Create the coefficient matrix of the equations coresponding to the segments

In [ ]:
M = sqr.matrix()
print("{} --det--> {}".format(M, np.linalg.det(M)))

Connecting $\pm 1$'s on the matrix

In [ ]:
fig = get_diagram(M, figsize=(500,500))
fig.show()

## All in One

In [ ]:
sqr = square_random_generator(max=6)
# sqr = Square((2,1,0,1))
M = sqr.matrix()

output = "{}\n\n {} |--det--> {}\n\n".format(sqr.code, M, np.linalg.det(M))
print(output)

figs = [get_diagram(M), sqr.graphic()]
for i in range(len(figs)): figs[i].show()

In [ ]:
print(figs[0])

## Observation on the incidenceMatrices

### Cofactor expansion 

In [ ]:
def cofexp_raw(sqr_matrix, raw_num=0):
    expansion = []
    raw = sqr_matrix[raw_num]
    for j in range(raw.size):
        if raw[j] != 0:
            shrinked = np.delete(np.delete(sqr_matrix, raw_num, 0),j,1)
            sign = (-1)**(raw_num+j)
            expansion.append([sign*raw[j], shrinked])
    return expansion

__予想1:__ 接続行列のどの行(最終行を除く)で余因子展開しても、
全体が正則なら、必ず行列式 $\pm 1$ のものが一つだけ。

これが正しいなら、正則な接続行列の行列式は $\pm 1$ であるとも言える。

__予想2:__ 同様の余因子展開について、非正則ならすべて行列式 $0$.

こちらの予想は反例があった。ex) Square(9,7,9,7), Square(8,4,6,2)

In [ ]:
#sqr = square_random_generator(); #Square((0,3,0,3))
#M = sqr.matrix()

print("{} --det--> {}\n".format(sqr.code, np.linalg.det(M)))
# print("{}\n".format(M))

for rn in range(len(sqr.segments)):
    expn = cofexp_raw(M, raw_num=rn)
    values = []
    for pair in expn:
        m = pair[1]
        values.append(int(np.linalg.det(m)))
    else: print("raw {} --raw_exp--> {}".format(rn, values))

__予想2(修正):__ 同様の余因子展開について、非正則なら、どの行でも 2つの行列式を足すと $0\mod 2$.


In [ ]:
def recursive_cofexp_raw(coeff_matrix_pair, count=2):
    if count > 0:
        count += -1
        sqr_matrix = coeff_matrix_pair[1]
#         print(sqr_matrix)
        coeff_matrix_pair[1] = [recursive_cofexp_raw(pair, count) for pair in cofexp_raw(sqr_matrix)]
    return coeff_matrix_pair

In [ ]:
expansion = recursive_cofexp_raw([1,M], count=5)
# print(expansion)

def show_expansion(expn, depth=0):
    for v in expn:
        if isinstance(v, list):
            show_expansion(v, depth+1)
        else:
            if isinstance(v, np.ndarray):
                det = np.linalg.det(v)
                if det != 0:
                    print("{} --> {}".format(v, np.linalg.det(v)))

show_expansion(expansion, depth=3)            

### Elementary deformations

In [ ]:
def elem_matrix(size=1, position=(1,1)):
    i,j = position
    P = np.eye(size, size, dtype=int)
    P[i][j] = 1
    return P
def P(n,ij):
    return elem_matrix(size=n, position=ij)

deform an incidence matrix by elementary matrices

In [ ]:
sqr = square_random_generator(max=5)
s = len(sqr.segments)

N = copy(sqr.matrix())

num = 0
for side in sqr.sides.values():
    for k in range(side.number):
            N = N@P(s+1,(num+k+1, num+0))
            N = P(s+1,(num+0, num+k+1))@N
    num += side.number

print(N)

### Others

In [ ]:
sqr = square_random_generator(verbose=False) #Square((0,1,1,0)) #
s = len(sqr.segments)

N = copy(sqr.matrix())
det0 = np.linalg.det(N)
N[s][s]=1
det1 = np.linalg.det(N)
print("{} --det0--> {} --det1--> {}".format(sqr.code, det0, det1))

__予想 (Others):__ 右下を1にした行列の行列式は、もとの2倍である。